# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Measure-Dynamic-Functional-Connectivity" data-toc-modified-id="Measure-Dynamic-Functional-Connectivity-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Measure Dynamic Functional Connectivity</a></div><div class="lev2 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Gather-all-data-sets" data-toc-modified-id="Gather-all-data-sets-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Gather all data sets</a></div><div class="lev2 toc-item"><a href="#Generate-processing-list" data-toc-modified-id="Generate-processing-list-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Generate processing list</a></div><div class="lev2 toc-item"><a href="#Submit-proc_list-to-qsub-as-jobs" data-toc-modified-id="Submit-proc_list-to-qsub-as-jobs-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Submit proc_list to qsub as jobs</a></div><div class="lev2 toc-item"><a href="#Submit-proc_list-to-qsub-as-jobs" data-toc-modified-id="Submit-proc_list-to-qsub-as-jobs-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Submit proc_list to qsub as jobs</a></div><div class="lev2 toc-item"><a href="#Continuously-monitor-queue-size-and-run-remaining-jobs" data-toc-modified-id="Continuously-monitor-queue-size-and-run-remaining-jobs-16"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Continuously monitor queue size and run remaining jobs</a></div><div class="lev1 toc-item"><a href="#Check-Adjacency" data-toc-modified-id="Check-Adjacency-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Check Adjacency</a></div>

# Measure Dynamic Functional Connectivity

## Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob
import time

import pickle as pkl
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/home/akhambhati/JagHome/hoth_research/Echobase')
import Echobase

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_CoreData = '/data/jag/akhambhati/Remotes/CORE.RAM_Stim'
path_PeriphData = '/home/akhambhati/Procd_Data/RSRCH.RAM_Stim'
path_ExpData = path_PeriphData + '/e01b-FuncNetw'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

## Gather all data sets

In [ ]:
path_list = glob.glob('{}/Baseline_Trials/*.mat'.format(path_CoreData))

data_list = []
for pth in path_list:    
    full_file = pth.split('/')[-1]    
    subj_id = full_file.split('.')[0]
    trial_id = full_file.split('.')[-2]

    chan_file = glob.glob('{}/Exp_Info/Channel_Info/{}.mat'.format(path_CoreData, subj_id))
    if len(chan_file) != 1:
        continue
    chan_file = chan_file[0]

    event_file = glob.glob('{}/Exp_Info/PS_Events/{}_events.mat'.format(path_CoreData, subj_id))
    if len(event_file) != 1:
        continue
    event_file = event_file[0]

    trial_lut_file = glob.glob('{}/Exp_Info/LUT_Trial_Events/{}_trial_lut.mat'.format(path_CoreData, subj_id))
    if len(trial_lut_file) != 1:
        continue
    trial_lut_file = trial_lut_file[0]
    
    data_list.append({'chan_path': chan_file,
                      'event_path': event_file,
                      'trial_lut_path': trial_lut_file,
                      'trial_id': trial_id.split('_')[-1],
                      'raw_path': pth,
                      'raw_name': '{}.{}'.format(subj_id, trial_id)})

## Generate processing list

In [ ]:
proc_item = []
for d_item in data_list:
    chan_path = d_item['chan_path']
    event_path = d_item['event_path']
    trial_lut_path = d_item['trial_lut_path']  
    trial_id = d_item['trial_id']
    raw_path = d_item['raw_path']
    raw_name = d_item['raw_name']
    res_path = '{}/Adjacency.{}.npz'.format(path_ExpData, raw_name)

    if os.path.exists(res_path):
        continue

    if os.path.exists('{}/Adjacency.{}.stdout'.format(path_ExpData, raw_name)):
        continue

    proc_item.append({'chan_path': chan_path,
                      'event_path': event_path,
                      'trial_lut_path': trial_lut_path,     
                      'trial_id': trial_id,
                      'raw_path': raw_path,
                      'raw_name': raw_name,
                      'res_path': res_path})
print('There are {} jobs to process.'.format(len(proc_item)))
    
pkl.dump(proc_item, open('{}/proc_list.pkl'.format(path_ExpData), 'w'))  

## Submit proc_list to qsub as jobs

In [ ]:
!qsub -t 1-1 ./e01-Measure_Adjacency-Baseline.py

# Check Adjacency

In [ ]:
%matplotlib inline

# Check fin_items for a list of processed adjacency matrices
adj_paths = np.random.permutation(glob.glob('{}/*.npz'.format(path_ExpData)))
adj = np.load(adj_paths[-1])['adj'].item()

print(adj_paths[-1])
print('No-Stimulus Adjacency')
plt.figure(figsize=(3,3), dpi=200)
ax = plt.subplot(2,2,1); ax.matshow(adj['No_Stim']['AlphaTheta'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,2); ax.matshow(adj['No_Stim']['Beta'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,3); ax.matshow(adj['No_Stim']['LowGamma'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,4); ax.matshow(adj['No_Stim']['HighGamma'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
plt.show()